In [1]:
from pytorch_libraries.model_NeuralStyleTransfer import *
from IPython.display import clear_output
from pylab import *
%matplotlib inline

In [2]:
# Image preprocessing
# For normalization, see https://github.com/pytorch/vision#models
transform = transforms.Compose([transforms.Resize(256),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), 
                                                     (0.229, 0.224, 0.225))
                               ]
                              )


content = Variable(load_image('../../data/images/chicago2.jpg', transform ))
style   = Variable(load_image('../../data/images/v1.jpg', transform, shape = [content.size(3), content.size(2)] ))

image   = Variable(content.data.clone(), requires_grad=True)

vgg     = VGGNet().cuda()

In [3]:
optimizer = torch.optim.Adam([image], lr=1e-2)
STYLE_WEIGHT  = 1000.0
TOTAL_OPTSTEP = 100

before = image.clone().cpu().data.clamp_(0, 1)

In [4]:
for step in range(TOTAL_OPTSTEP):
        
        # Extract multiple(5) conv feature vectors
        image_features   = vgg(image)
        content_features = vgg(content)
        style_features   = vgg(style)

        style_loss = 0
        content_loss = 0
        
        # for each layer
        lw = [0,0,1,10,100]
        for i, fi in enumerate(image_features):
            fc = content_features[i]
            fs = style_features[i]
            
            # Compute content loss (target and content image)
            content_loss += torch.mean((fi - fc)**2)
            
            # Reshape conv features
            _, c, h, w = fi.size()
            fi = fi.view(c, h * w)
            fs = fs.view(c, h * w)
            # Compute gram matrix  
            grami = torch.mm(fi, fi.t())
            grams = torch.mm(fs, fs.t())
            # Compute style loss (target and style image)
            style_loss += torch.mean((grami - grams)**2) / (c * h * w) * STYLE_WEIGHT #* lw[i] / (c * h * w) 
            
        # Compute total loss, backprop and optimize
        #loss = content_loss + style_loss 
        loss =  style_loss 
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
                 
        if (step+1) % 10 == 0:
            after = image.clone().cpu().data.clamp_(0, 1)
            print ('Step [%d/%d], Content Loss: %.4f, Style Loss: %.4f, change %.4f' 
                   %(step+1, TOTAL_OPTSTEP, content_loss.data[0], style_loss.data[0], torch.mean(torch.abs(after-before))))
            before = after
    
        if (step+1) % 10 == 0:
            # Save the generated image
            denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
            img = image.clone().cpu().squeeze()
            img = denorm(img.data).clamp_(0, 1)
            torchvision.utils.save_image(img, '../../plot/nst/output-%d.png' %(step+1))
        

Step [10/100], Content Loss: 16.6283, Style Loss: 159701.0156, change 0.0224
Step [20/100], Content Loss: 22.8062, Style Loss: 68588.5391, change 0.0159
Step [30/100], Content Loss: 26.6445, Style Loss: 42595.1445, change 0.0106
Step [40/100], Content Loss: 27.5927, Style Loss: 29938.7266, change 0.0071
Step [50/100], Content Loss: 28.9646, Style Loss: 22989.2988, change 0.0052
Step [60/100], Content Loss: 29.7151, Style Loss: 18821.1777, change 0.0040
Step [70/100], Content Loss: 30.5004, Style Loss: 16019.4629, change 0.0033
Step [80/100], Content Loss: 30.9950, Style Loss: 13982.4746, change 0.0028
Step [90/100], Content Loss: 31.4746, Style Loss: 12439.2998, change 0.0025
Step [100/100], Content Loss: 31.9156, Style Loss: 11225.8037, change 0.0023


In [5]:
 models.vgg19(pretrained=True).features

Sequential (
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU (inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU (inplace)
  (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU (inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU (inplace)
  (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU (inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU (inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU (inplace)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU (inplace)
  (18): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (19): Conv2d(256,